<a href="https://colab.research.google.com/github/Sukitha2808/Data_Science_GColab/blob/main/Data_Engineering_with_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import timedelta

# --- 1. MOCK DATA CREATION ---
# Since we don't have the actual data, we'll create mock DataFrames
# based on the schemas you provided.

# Shipments Table
shipments_df = pd.DataFrame({
    'shipment_id': ['S1', 'S2', 'S3', 'S4', 'S5', 'S6'],
    'origin_warehouse': ['A', 'B', 'A', 'C', 'B', 'C'],
    'destination_city': ['New York', 'Los Angeles', 'Chicago', 'New York', 'Chicago', 'Houston'],
    'ship_date': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06']),
    'delivery_date': pd.to_datetime(['2023-01-05', '2023-01-07', '2023-01-08', '2023-01-04', '2023-01-10', '2023-01-09']),
    'product_id': ['P1', 'P2', 'P1', 'P3', 'P4', 'P2'],
    'quantity': [10, 20, 15, 50, 5, 25],
    'freight_cost': [100.50, 250.75, 120.00, 300.25, 80.00, 280.50]
})

# Delivery Logs Table
delivery_logs_df = pd.DataFrame({
    'delivery_id': ['D1', 'D2', 'D3', 'D4', 'D5', 'D6'],
    'shipment_id': ['S1', 'S2', 'S3', 'S4', 'S5', 'S6'],
    'carrier': ['FedEx', 'UPS', 'FedEx', 'DHL', 'UPS', 'FedEx'],
    'status': ['Delivered', 'Delivered', 'Delivered', 'Delayed', 'Delivered', 'Delivered'],
    'delivery_duration_days': [4, 5, 5, 0, 5, 3],
    'damage_flag': [False, False, True, False, False, True],
    'proof_of_delivery_status': ['Received', 'Received', 'Received', 'Not Available', 'Received', 'Received']
})

# Claims Table
claims_df = pd.DataFrame({
    'claim_id': ['C1', 'C2'],
    'delivery_id': ['D3', 'D6'],
    'reason': ['Damaged in transit', 'Missing items'],
    'amount_claimed': [150.00, 200.00],
    'claim_status': ['Resolved', 'Pending'],
    'claim_date': pd.to_datetime(['2023-01-09', '2023-01-11']),
    'resolved_date': pd.to_datetime(['2023-01-15', None]) # 'None' for unresolved claim
})

# Inventory Table (for reorder flag calculation)
inventory_df = pd.DataFrame({
    'warehouse_id': ['A', 'B', 'C'],
    'product_id': ['P1', 'P2', 'P3'],
    'stock_level': [50, 10, 5],
    'reorder_threshold': [75, 15, 10]
})

# Vendors Table (for grouped summaries in SQL section)
vendors_df = pd.DataFrame({
    'vendor_id': ['V1', 'V2', 'V3'],
    'vendor_name': ['Vendor A', 'Vendor B', 'Vendor C'],
    'product_id': ['P1', 'P2', 'P3']
})

# --- 2. PYTHON WITH PANDAS TASKS ---

# Task: Clean and merge shipment, delivery, and claim data.
# First, merge Shipments and Delivery Logs on 'shipment_id'.
merged_df = pd.merge(shipments_df, delivery_logs_df, on='shipment_id', how='left')

# Then, merge the result with Claims on 'delivery_id'.
final_df = pd.merge(merged_df, claims_df, on='delivery_id', how='left')

# Drop the redundant 'shipment_id' from the claims table if it existed, for clarity.
final_df = final_df.drop(columns=['shipment_id'])

print("--- Merged DataFrame ---")
print(final_df)
print("\n")

# Task: Calculate delay duration, reorder flags, claim aging.

# o Calculate delay duration.
# This is the difference between the actual delivery date and the ship date.
final_df['delay_duration_days'] = (final_df['delivery_date'] - final_df['ship_date']).dt.days

# o Calculate reorder flags.
# We need to join with the Inventory table to get stock levels.
# First, let's group our shipment data to get the total quantity shipped per product.
shipment_summary = final_df.groupby(['origin_warehouse', 'product_id'])['quantity'].sum().reset_index()
shipment_summary.rename(columns={'quantity': 'shipped_quantity'}, inplace=True)

# Merge with the inventory data.
inventory_merged = pd.merge(inventory_df, shipment_summary,
                            left_on=['warehouse_id', 'product_id'],
                            right_on=['origin_warehouse', 'product_id'],
                            how='left')
inventory_merged['shipped_quantity'] = inventory_merged['shipped_quantity'].fillna(0) # Fill NaN with 0

# Calculate new stock level after shipments.
inventory_merged['current_stock'] = inventory_merged['stock_level'] - inventory_merged['shipped_quantity']

# Create the reorder flag.
inventory_merged['reorder_flag'] = inventory_merged['current_stock'] < inventory_merged['reorder_threshold']

print("--- Inventory with Reorder Flags ---")
print(inventory_merged[['warehouse_id', 'product_id', 'current_stock', 'reorder_threshold', 'reorder_flag']])
print("\n")

# o Calculate claim aging.
# This is the time between the claim being made and it being resolved.
final_df['claim_aging_days'] = (final_df['resolved_date'] - final_df['claim_date']).dt.days

print("--- Final DataFrame with Calculated Metrics ---")
print(final_df[['shipment_id', 'delay_duration_days', 'claim_aging_days']])